In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import nfl_data_py as nfl
import numpy as np

In [2]:
def rushing_yards_allowed(team: str, year: int) -> pd.DataFrame:
    df_plays=nfl.import_pbp_data([year])

    df_team=df_plays[df_plays['defteam'] == team]
    
    df_team = df_team.drop_duplicates(subset=['game_id', 'play_id'])
    
    rushing=df_team[df_team['play_type']=='run'].groupby(['defteam','posteam', 'week'])['rushing_yards'].sum().reset_index(name='rushing_yards_allowed')

    return rushing.sort_values(by=['defteam', 'week'])     
    

In [3]:
def passing_yards_allowed(team: str, year: int) -> pd.DataFrame:
    df_plays=nfl.import_pbp_data([year])

    df_team=df_plays[df_plays['defteam'] == team]
    
    df_team = df_team.drop_duplicates(subset=['game_id', 'play_id'])

    passing=df_team[df_team['play_type']=='pass'].groupby(['defteam','posteam', 'week'])['passing_yards'].sum().reset_index(name='passing_yards_allowed')

    return passing.sort_values(by=['defteam', 'week']) 

In [4]:
def touchdowns_allowed(team: str, year: int) -> pd.DataFrame:
    df_plays=nfl.import_pbp_data([year])

    df_team=df_plays[df_plays['defteam'] == team]
    
    df_team = df_team[df_team['play_type'].isin(['run', 'pass'])]

    df_team = df_team[df_team['touchdown'] == True]
    
    df_team = df_team.drop_duplicates(subset=['game_id', 'play_id'])

    tds = df_team.groupby(['defteam', 'posteam', 'week']).size().reset_index(name='touchdowns_allowed')
        

    return tds.sort_values(by=['defteam', 'week']) 

In [5]:
def turnovers_created(team: str, year: int) -> pd.DataFrame:
    df_plays=nfl.import_pbp_data([year])

    df_team=df_plays[df_plays['defteam'] == team]
    
    df_turnovers = df_team[(df_team['interception'] == True) |(df_team['fumble'] == True)] 
    
    df_turnovers = df_turnovers.drop_duplicates(subset=['game_id', 'play_id'])

    turnovers = df_turnovers.groupby(['defteam', 'posteam', 'week']).size().reset_index(name='turnovers_created')
        

    return turnovers.sort_values(by=['defteam', 'week']) 

In [6]:
def turnovers_recovered(team: str, year: int) -> pd.DataFrame:
    df_plays=nfl.import_pbp_data([year])

    df_team=df_plays[df_plays['defteam'] == team]
    
    df_turnovers = df_team[(df_team['interception'] == True) |((df_team['fumble'] == True) & (df_team['fumble_lost'] == True))]
    
    df_turnovers = df_turnovers.drop_duplicates(subset=['game_id', 'play_id'])

    turnovers = df_turnovers.groupby(['defteam', 'posteam', 'week']).size().reset_index(name='turnovers_created')
        

    return turnovers.sort_values(by=['defteam', 'week']) 

In [7]:
def red_zone_percentage(team: str, year:int) -> pd.DataFrame:
    
    df_plays=nfl.import_pbp_data([year])

    df_team=df_plays[df_plays['defteam'] == team]

    df_red=df_team[df_team['yardline_100'] <= 20] 
    
    df_red = df_red.drop_duplicates(subset=['game_id', 'play_id'])

    redzone_plays = df_red.groupby(['defteam', 'posteam', 'week']).size().reset_index(name='redzone_plays')
    
    tds_red = df_red[df_red['touchdown'] == True]
    
    redzone_tds = tds_red.groupby('week').size().reset_index(name='touchdowns_allowed_red')

    summary = pd.merge(redzone_plays, redzone_tds, on='week', how='left')
    summary['touchdowns_allowed_red'] = summary['touchdowns_allowed_red'].fillna(0)

    summary['red_zone_td_pct'] = summary['touchdowns_allowed_red'] / summary['redzone_plays']


    return summary.sort_values(by=['defteam', 'week'])

    